<a href="https://colab.research.google.com/github/thisisSHAX/ML_Ops_Project-/blob/main/Autism_Screening_Prediction_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Creating a web app to deploy the model

In [1]:
!pip install wandb
!pip install onnxruntime
!pip install -q gradio 

     |████████████████████████████████| 1.7 MB 29.2 MB/s 
     |████████████████████████████████| 140 kB 56.2 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 180 kB 49.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=1f6fccb5bc2a10cc9a6f4b790a2f32ba40fd880e09ca7962d39aea1158f95b93
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=7eab7cb6c7a2befc24964912953041ec7cefc329ed50c3b8bb97cfa94c87eca6
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
     |████████████████████████████████| 4.8 MB 29.1 MB/s 
     |████████████████████████████████| 979 kB 30.0 MB/s 
     |████████████████████

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import os
import wandb
import pandas as pd
import numpy as np
import onnxruntime as rt
from gradio import gradio as gr

### Initialise weights and biases

In [4]:
os.environ["WANDB_API_KEY"] = "26ff855733a63e9d999357d3e3bd1eb135cff5a5"

In [5]:
run = wandb.init(project='autism_screening_deploy')

wandb: Currently logged in as: thisisshax (use `wandb login --relogin` to force relogin)


### Download the model and load it

In [6]:
ARTIFACT_NAME = 'LogReg_AutismScreening:v0'

In [7]:
artifact = run.use_artifact('maytune/autism_screening/' + ARTIFACT_NAME, type='model')
artifact_dir = artifact.download()

In [8]:
artifact_dir

'./artifacts/LogReg_AutismScreening:v0'

In [9]:
!ls -al  artifacts/LogReg_AutismScreening:v0

total 16
drwxr-xr-x 2 root root 4096 Nov 25 14:10 .
drwxr-xr-x 3 root root 4096 Nov 25 14:10 ..
-rw-r--r-- 1 root root 5380 Nov 25 14:10 autismscreening_logreg.onnx


### Columns

In [10]:
x_columns = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','Age_Mons','Sex','Ethnicity','Jaundice','Family_mem_with_ASD']

In [11]:
cat_features = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','Sex','Ethnicity','Jaundice','Family_mem_with_ASD']

In [12]:
num_features = list(set(x_columns) - set(cat_features))

### Implement the predict() function

In [13]:
def predict_autism(A1, A2, A3,
                  A4, A5, A6, 
                  A7, A8, A9, 
                  A10, Age_Mons, Sex, Ethnicity, Jaundice, Family_mem_with_ASD ):

    inputs_dict = {'A1' :np.float32(A1) , 
              'A2': np.float32(A2), 
              'A3': np.float32(A3), 
              'A4': np.float32(A4), 
              'A5': np.float32(A5), 
              'A6': np.float32(A6), 
              'A7': np.float32(A7), 
              'A8': np.float32(A8), 
              'A9': np.float32(A9), 
              'A10': np.float32(A10), 
              'Age_Mons' : float(Age_Mons),
              'Sex': Sex, 
              'Ethnicity': Ethnicity,
              'Jaundice': Jaundice,
              'Family_mem_with_ASD': Family_mem_with_ASD}

    df = pd.DataFrame(inputs_dict, index = [0])
    print(df)

    inputs = {c: df[c].values for c in df.columns}
    for c in num_features:
        inputs[c] = inputs[c].astype(np.float32)
    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))            
  
    sess = rt.InferenceSession(artifact_dir + '/autismscreening_logreg.onnx')
    pred_onx = sess.run(None, inputs)
    predicted = pred_onx[0][0]
   # predicted = pred_onx[0][0,0]
    

   # print(pred_onx)
  
    return {'Autism prediction: ' + str(predicted)}

### Implement UI

In [ ]:
Age_Mons = gr.inputs.Slider(minimum=0, maximum=40, label="Age of infant (in months)")

A1 = gr.inputs.Radio([0, 1], default=0, label="Does your child look at you when you call his/her name?")
A2 = gr.inputs.Radio([0, 1], default=0, label="Is it easy for you to get eye-contact with your child?")
A3 = gr.inputs.Radio([0, 1], default=0, label="Does your child point to indicate that he/she wants something?")
A4 = gr.inputs.Radio([0, 1], default=0, label="Does your child point to share interest with you?")
A5 = gr.inputs.Radio([0, 1], default=0, label="Does your child pretend?")
A6 = gr.inputs.Radio([0, 1], default=0, label="Does your child follow where you are looking?")
A7 = gr.inputs.Radio([0, 1], default=0, label="If you or someone else in your family is visibly upset, Does your child show signs of wanting to comfort them?")
A8 = gr.inputs.Radio([0, 1], default=0, label="Does your child use simple words?")
A9 = gr.inputs.Radio([0, 1], default=0, label="Does your child use simple gestures")
A10 = gr.inputs.Radio([0, 1], default=0, label="Does your child stare at nothing with no apparent purpose?")

Sex = gr.inputs.Radio(['m', 'f'], default="m", label="Sex")
Ethnicity = gr.inputs.Dropdown(['asian', 'black', 'Hispanic', 'Latino', 'middle eastern', 'mixed', 'Native Indian', 'Others', 'Pacifica', 'south asian', 'White European'], default="Others", label="Ethnicity")
Jaundice = gr.inputs.Radio(['no', 'yes'], default="no", label="Was the child born with Jaundice?")
Family_mem_with_ASD = gr.inputs.Radio(['no', 'yes'], default="no", label="Does any immediate family member have ASD?")

gr.Interface(predict_autism, [A1, A2, A3,
                  A4, A5, A6, 
                  A7, A8, A9, 
                  A10, Age_Mons, Sex, Ethnicity, Jaundice, Family_mem_with_ASD], "text", live=False).launch(debug=True);

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://31244.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


    A1   A2   A3   A4   A5  ...  Age_Mons  Sex  Ethnicity  Jaundice  Family_mem_with_ASD
0  1.0  1.0  1.0  0.0  0.0  ...       2.3    m      asian        no                   no

[1 rows x 15 columns]
    A1   A2   A3   A4   A5  ...  Age_Mons  Sex  Ethnicity  Jaundice  Family_mem_with_ASD
0  1.0  1.0  1.0  0.0  0.0  ...       2.3    m      asian        no                  yes

[1 rows x 15 columns]
